Je souhaite travailler sur le fichier G_2047_Details.csv pour avoir des statistiques sur l'agriculture française en générale, afin de pouvoir remettre en contexte celles de l'agriculture biologique et de lui donner son juste poids.

J'ai téléchargé la base de données directement sur mon dépôt Git (après maint essais différents pour lire une base de données trop volumineuse) et j'essaie maintenant de la lire

Je ne peux lire la base de données directement car il semblerait qu'il y ait une erreur de séparateur ou autre. Tentons de résoudre cette erreur.

Le fichier que j'essaie de lire contient des lignes descriptives en haut (informations générales) qui ne font pas partie des données tabulaires. Ces lignes doivent être ignorées pour charger correctement les données dans Pandas.

En regardant le contenu brut, les données tabulaires semblent commencer à la ligne contenant les colonnes (ici : "Superficie agricole utilisée";"Nombre d'exploitations concernées";...).

Pour résoudre le problème, nous allons ignorer les lignes précédentes (mais nous copions le contenu de ces lignes ci dessous afin de conserver leur contenu).

Le document est un tableau à double entrée, j'analyse donc la visualisation disponible sur le site [Agreste](https://agreste.agriculture.gouv.fr/agreste-saiku/?plugin=true&query=query/open/G_2047#query/open/G_2047) pour mieux savoir comment travailler avec. Je remarque que seulement la première ligne m'intéresse puisqu'elle correspond au total des suivantes, qui subdivise les exploitations agricoles par catégorie de surface agricole utilisée.

Je sépare maintenant mon tableau en plusieurs tableaux correspondants à chacune des grandes colonnes de mon tableau mère (car chacune de ces colonnes comprend ensuite des sous colonnes en fonction de tous les départements).

Effectuons le même travail à présent sur le fichier 'G_2047_Details.csv' car c'est grâce à celui ci qu'on va obtenir nos statistiques.

In [32]:
# Lire le fichier brut pour inspecter son contenu
with open("./G_2047_Details.csv", "r", encoding="utf-8") as f:
    content = f.readlines()
    print("Contenu brut du fichier (10 premières lignes) :")
    print("".join(content[:10]))


Contenu brut du fichier (10 premières lignes) :
"Exploitations par tranche de superficie agricole utilisée (SAU)"
""
"Source";"Agreste - Recensements"
"notes";"- Les données sont localisées au siège de l'exploitation agricole. Les années 1970 et 1979 ne portent que sur la France métropolitaine."
""
"//Superficie agricole utilisée";"Nombre d'exploitations concernées/1970/971 - Guadeloupe";"Nombre d'exploitations concernées/1970/972 - Martinique";"Nombre d'exploitations concernées/1970/973 - Guyane";"Nombre d'exploitations concernées/1970/974 - La Réunion";"Nombre d'exploitations concernées/1970/75 - Ville de Paris";"Nombre d'exploitations concernées/1970/77 - Seine-et-Marne";"Nombre d'exploitations concernées/1970/78 - Yvelines";"Nombre d'exploitations concernées/1970/91 - Essonne";"Nombre d'exploitations concernées/1970/92 - Hauts-de-Seine";"Nombre d'exploitations concernées/1970/93001 - Aubervilliers";"Nombre d'exploitations concernées/1970/93005 - Aulnay-sous-Bois";"Nombre d'exploita

Mon nouveau fichier est beaucoup plus complexe car il comporte des en-têtes multiples. Essayons de les traiter.

In [33]:
import pandas as pd

# Chemin du fichier
file_path = "./G_2047_Details.csv"

# Identifier où commencent les données utiles
start_line = 5  # Ligne à partir de laquelle les données utiles commencent (ajustez si nécessaire)

# Charger les données avec en-têtes multiples
data = pd.read_csv(file_path, sep=";", encoding="utf-8", skiprows=start_line, header=[0])

# Vérifier un aperçu des données
print("Aperçu des données chargées :")
data.head()

Aperçu des données chargées :


,//Superficie agricole utilisée,Nombre d'exploitations concernées/1970/971 - Guadeloupe,Nombre d'exploitations concernées/1970/972 - Martinique,Nombre d'exploitations concernées/1970/973 - Guyane,Nombre d'exploitations concernées/1970/974 - La Réunion,Nombre d'exploitations concernées/1970/75 - Ville de Paris,Nombre d'exploitations concernées/1970/77 - Seine-et-Marne,Nombre d'exploitations concernées/1970/78 - Yvelines,Nombre d'exploitations concernées/1970/91 - Essonne,Nombre d'exploitations concernées/1970/92 - Hauts-de-Seine,...,Production brute standard (millier d'euros)/2010/73 - Savoie,Production brute standard (millier d'euros)/2010/74 - Haute-Savoie,Production brute standard (millier d'euros)/2010/04 - Alpes-de-Haute-Provence,Production brute standard (millier d'euros)/2010/05 - Hautes-Alpes,Production brute standard (millier d'euros)/2010/06 - Alpes-Maritimes,Production brute standard (millier d'euros)/2010/13 - Bouches-du-Rhône,Production brute standard (millier d'euros)/2010/83 - Var,Production brute standard (millier d'euros)/2010/84 - Vaucluse,Production brute standard (millier d'euros)/2010/2A - Corse-du-Sud,Production brute standard (millier d'euros)/2010/2B - Haute-Corse
0,Ensemble,NaN,NaN,NaN,NaN,32.0,5642,3146,2523,167.0,...,249710,329259,263404,171695,183106,889243,841261,1360173,78163,286813
1,"SAU inférieure à 5 ha, y compris sans SAU",NaN,NaN,NaN,NaN,21.0,1142,1169,980,154.0,...,30271,38168,12401,11221,117902,135040,214263,93020,7107,18496
2,SAU de 5 à moins de 20 ha,NaN,NaN,NaN,NaN,4.0,611,711,391,10.0,...,43367,33551,22972,21266,28699,231682,276606,378208,10168,44668
3,SAU de 20 à moins de 35 ha,NaN,NaN,NaN,NaN,NaN,718,354,252,NaN,...,22736,27359,26993,22694,18757,118662,108708,319729,9809,41778
4,SAU de 35 à moins de 50 ha,NaN,NaN,NaN,NaN,NaN,697,216,225,NaN,...,17101,37599,20556,22157,1223,63899,74352,206512,9043,34130


Filtrons le tableau su la première ligne "Ensemble" qui correspond au total des lignes suivantes séparant les exploitations agricoles selon leur taille.

In [34]:
# Filtrer uniquement la ligne correspondant à "Ensemble" dans la première colonne
filtered_data = data[data.iloc[:, 0].apply(str) == "Ensemble"]

# Vérifier les dimensions après le filtrage
print(f"Lignes après filtrage : {filtered_data.shape[0]}")
filtered_data.head()

Lignes après filtrage : 1


,//Superficie agricole utilisée,Nombre d'exploitations concernées/1970/971 - Guadeloupe,Nombre d'exploitations concernées/1970/972 - Martinique,Nombre d'exploitations concernées/1970/973 - Guyane,Nombre d'exploitations concernées/1970/974 - La Réunion,Nombre d'exploitations concernées/1970/75 - Ville de Paris,Nombre d'exploitations concernées/1970/77 - Seine-et-Marne,Nombre d'exploitations concernées/1970/78 - Yvelines,Nombre d'exploitations concernées/1970/91 - Essonne,Nombre d'exploitations concernées/1970/92 - Hauts-de-Seine,...,Production brute standard (millier d'euros)/2010/73 - Savoie,Production brute standard (millier d'euros)/2010/74 - Haute-Savoie,Production brute standard (millier d'euros)/2010/04 - Alpes-de-Haute-Provence,Production brute standard (millier d'euros)/2010/05 - Hautes-Alpes,Production brute standard (millier d'euros)/2010/06 - Alpes-Maritimes,Production brute standard (millier d'euros)/2010/13 - Bouches-du-Rhône,Production brute standard (millier d'euros)/2010/83 - Var,Production brute standard (millier d'euros)/2010/84 - Vaucluse,Production brute standard (millier d'euros)/2010/2A - Corse-du-Sud,Production brute standard (millier d'euros)/2010/2B - Haute-Corse
0,Ensemble,NaN,NaN,NaN,NaN,32.0,5642,3146,2523,167.0,...,249710,329259,263404,171695,183106,889243,841261,1360173,78163,286813


Nous cherchons maintenant à supprimer la première colonne qui ne sert plus à rien et nous cherchons à subdiviser le tableau en deux tableaux, l'un portant uniquement sur le nombre d'exploitations agricoles concernées et l'autre sur la superficie exploitée, les deux tableaux étant subdivisés par années de références et par département. 

In [35]:
# Supprimer la première colonne
filtered_data = filtered_data.iloc[:, 1:]

# Subdiviser le tableau en deux nouveaux tableaux
# Tableau avec les colonnes dont l'en-tête contient "Nombre d'exploitations concernées"
exploitations_data = filtered_data.filter(like="Nombre d'exploitations concernées")

# Tableau avec les colonnes dont l'en-tête contient "Superficie agricole utilisée (ha)"
superficie_data = filtered_data.filter(like="Superficie agricole utilisée (ha)")

# Vérifier les nouveaux tableaux
print("\nTableau des exploitations :")
print(exploitations_data.head())

print("\nTableau des superficies :")
superficie_data.head()


Tableau des exploitations :
   Nombre d'exploitations concernées/1970/971 - Guadeloupe  \
0                                                NaN         

   Nombre d'exploitations concernées/1970/972 - Martinique  \
0                                                NaN         

   Nombre d'exploitations concernées/1970/973 - Guyane  \
0                                                NaN     

   Nombre d'exploitations concernées/1970/974 - La Réunion  \
0                                                NaN         

   Nombre d'exploitations concernées/1970/75 - Ville de Paris  \
0                                               32.0            

   Nombre d'exploitations concernées/1970/77 - Seine-et-Marne  \
0                                               5642            

   Nombre d'exploitations concernées/1970/78 - Yvelines  \
0                                               3146      

   Nombre d'exploitations concernées/1970/91 - Essonne  \
0                                       

,Superficie agricole utilisée (ha)/1970/971 - Guadeloupe,Superficie agricole utilisée (ha)/1970/972 - Martinique,Superficie agricole utilisée (ha)/1970/973 - Guyane,Superficie agricole utilisée (ha)/1970/974 - La Réunion,Superficie agricole utilisée (ha)/1970/75 - Ville de Paris,Superficie agricole utilisée (ha)/1970/77 - Seine-et-Marne,Superficie agricole utilisée (ha)/1970/78 - Yvelines,Superficie agricole utilisée (ha)/1970/91 - Essonne,Superficie agricole utilisée (ha)/1970/92 - Hauts-de-Seine,Superficie agricole utilisée (ha)/1970/93001 - Aubervilliers,...,Superficie agricole utilisée (ha)/2010/73 - Savoie,Superficie agricole utilisée (ha)/2010/74 - Haute-Savoie,Superficie agricole utilisée (ha)/2010/04 - Alpes-de-Haute-Provence,Superficie agricole utilisée (ha)/2010/05 - Hautes-Alpes,Superficie agricole utilisée (ha)/2010/06 - Alpes-Maritimes,Superficie agricole utilisée (ha)/2010/13 - Bouches-du-Rhône,Superficie agricole utilisée (ha)/2010/83 - Var,Superficie agricole utilisée (ha)/2010/84 - Vaucluse,Superficie agricole utilisée (ha)/2010/2A - Corse-du-Sud,Superficie agricole utilisée (ha)/2010/2B - Haute-Corse
0,NaN,NaN,NaN,NaN,532.0,363644,105830,97128,536.0,NaN,...,232768,250598,290211,191793,83986,297141,133168,222436,125539,210252


Continuons de clarifier nos données en supprimant la mention de ce qui est mesuré dans l'en-tête de toutes les colonnes de nos tableaux car ceci est explicité dans le nom du dataset. 

Puis séparons nos tableaux en sous tableaux par année de référence. Pour cela nous créons Le type un dictionnaire (exploitations_by_year) où les clés sont les années et les valeurs sont des DataFrames pandas. Chaque DataFrame contient les données filtrées pour l'année correspondante.

In [ ]:
# # Fonction pour enlever l'indicateur des en-têtes
# def remove_indicator(df, indicator):
#     new_headers = [header.replace(indicator + "/", "") for header in df.columns]
#     df.columns = new_headers
#     return df

# # Enlever l'indicateur des en-têtes pour chaque tableau
# exploitations_data = remove_indicator(exploitations_data, "Nombre d'exploitations concernées")
# superficie_data = remove_indicator(superficie_data, "Superficie agricole utilisée (ha)")

# # Fonction pour séparer le tableau par année de référence et créer des variables dynamiques
# def separate_by_year_and_create_vars(df, prefix):
#     years = df.columns.str.split('/').str[0].unique()
#     separated_dfs = {year: df.filter(like=year) for year in years}
#     for year, df in separated_dfs.items():
#         df.attrs['name'] = f"{prefix}_{year}"  # Utiliser un attribut personnalisé pour stocker le nom
#         globals()[f"{prefix}_{year}"] = df
#     return separated_dfs

# # Séparer chaque tableau par année de référence et créer des variables dynamiques
# exploitations_by_year = separate_by_year_and_create_vars(exploitations_data, "exploitations")
# superficie_by_year = separate_by_year_and_create_vars(superficie_data, "superficie")


Pour encore plus de visibilités, nous souhaitons enlèver les dates des en-têtes des nouveaux tableaux séparés par année. Pour ce faire, nous réutilisons notre fonction `remove_indicateur` définie ci dessus.

In [ ]:
# # Appliquer la fonction remove_indicator aux sous-tableaux par année pour enlever les dates
# for year in exploitations_by_year:
#     exploitations_by_year[year] = remove_indicator(exploitations_by_year[year], year)

# for year in superficie_by_year:
#     superficie_by_year[year] = remove_indicator(superficie_by_year[year], year)


Je souhaite voir le contenu de mes dictionnaires.

In [ ]:
# # Fonction pour afficher le contenu des dictionnaires
# def display_dict_content(dictionary, name):
#     print(f"\nContenu du dictionnaire {name} :")
#     for key, df in dictionary.items():
#         print(f"\nClé : {key}")
#         print(f"Nom du DataFrame : {df.attrs['name']}")  # Récupérer le nom depuis l'attribut personnalisé

# # Afficher le contenu des dictionnaires
# display_dict_content(exploitations_by_year, "exploitations_by_year")
# display_dict_content(superficie_by_year, "superficie_by_year")


Contenu du dictionnaire exploitations_by_year :

Clé : 1970
Nom du DataFrame : exploitations_1970

Clé : 1979
Nom du DataFrame : exploitations_1979

Clé : 1988
Nom du DataFrame : exploitations_1988

Clé : 2000
Nom du DataFrame : exploitations_2000

Clé : 2010
Nom du DataFrame : exploitations_2010

Contenu du dictionnaire superficie_by_year :

Clé : 1970
Nom du DataFrame : superficie_1970

Clé : 1979
Nom du DataFrame : superficie_1979

Clé : 1988
Nom du DataFrame : superficie_1988

Clé : 2000
Nom du DataFrame : superficie_2000

Clé : 2010
Nom du DataFrame : superficie_2010


Je regarde à quoi ressemble mon tableau pour un dataframe afin de voir s'il reste des choses à modifier.

In [39]:
superficie_2010.head()

,971 - Guadeloupe,972 - Martinique,973 - Guyane,974 - La Réunion,75 - Ville de Paris,77 - Seine-et-Marne,78 - Yvelines,91 - Essonne,92 - Hauts-de-Seine,93001 - Aubervilliers,...,73 - Savoie,74 - Haute-Savoie,04 - Alpes-de-Haute-Provence,05 - Hautes-Alpes,06 - Alpes-Maritimes,13 - Bouches-du-Rhône,83 - Var,84 - Vaucluse,2A - Corse-du-Sud,2B - Haute-Corse
0,62802.0,49964,50690,85628.0,NaN,671720,178269,168289,26.0,NaN,...,232768,250598,290211,191793,83986,297141,133168,222436,125539,210252


Il reste de colonnes qui ne correspondent pas à des départements mais à des codes postaux. Il faut les supprimer pour n'avoir que les départements. On peut voir cette erreur avec le nombre de colonnes qui se monte à 179 alors qu'il y a autour de 100 département en France métropolitaine et d'outre mer. 

In [ ]:
# import re

# # Fonction pour supprimer les colonnes en fonction de la longueur des chiffres dans les en-têtes
# def remove_columns_by_length(df, max_length):
#     pattern = re.compile(r'^\d{4,}')  # Recherche des en-têtes commençant par 4 chiffres ou plus
#     filtered_columns = [col for col in df.columns if pattern.match(col.split(' - ')[0])]
#     return df.drop(columns=filtered_columns)

# # Appliquer la fonction pour supprimer les colonnes indésirables des DataFrames
# exploitations_data_cleaned = remove_columns_by_length(exploitations_data, 3)
# superficie_data_cleaned = remove_columns_by_length(superficie_data, 3)

# # Fonction pour séparer le tableau par année de référence et créer des variables dynamiques
# def separate_by_year_and_create_vars(df, prefix):
#     years = df.columns.str.split('/').str[0].unique()
#     separated_dfs = {year: df.filter(like=year) for year in years}
#     for year, df in separated_dfs.items():
#         df.attrs['name'] = f"{prefix}_{year}"  # Utiliser un attribut personnalisé pour stocker le nom
#         globals()[f"{prefix}_{year}"] = df
#     return separated_dfs

# # Séparer chaque tableau par année de référence et créer des variables dynamiques
# exploitations_by_year_cleaned = separate_by_year_and_create_vars(exploitations_data_cleaned, "exploitations")
# superficie_by_year_cleaned = separate_by_year_and_create_vars(superficie_data_cleaned, "superficie")

# # Appliquer la fonction remove_indicator aux sous-tableaux par année pour enlever les dates
# for year in exploitations_by_year_cleaned:
#     exploitations_by_year_cleaned[year] = remove_indicator(exploitations_by_year_cleaned[year], year)

# for year in superficie_by_year_cleaned:
#     superficie_by_year_cleaned[year] = remove_indicator(superficie_by_year_cleaned[year], year)

# # Fonction pour afficher le contenu des dictionnaires
# def display_dict_content(dictionary, name):
#     print(f"\nContenu du dictionnaire {name} :")
#     for key, df in dictionary.items():
#         print(f"\nClé : {key}")
#         print(f"Nom du DataFrame : {df.attrs['name']}")  # Récupérer le nom depuis l'attribut personnalisé

# # Afficher le contenu des dictionnaires
# display_dict_content(exploitations_by_year_cleaned, "exploitations_by_year_cleaned")
# display_dict_content(superficie_by_year_cleaned, "superficie_by_year_cleaned")

# # Exemple d'accès à un DataFrame en dehors du dictionnaire
# print("\nAccès direct à un DataFrame :")
# print(exploitations_1970.head())
# print(superficie_1970.head())




Contenu du dictionnaire exploitations_by_year_cleaned :

Contenu du dictionnaire superficie_by_year_cleaned :

Accès direct à un DataFrame :
   971 - Guadeloupe  972 - Martinique  973 - Guyane  974 - La Réunion  \
0               NaN               NaN           NaN               NaN   

   75 - Ville de Paris  77 - Seine-et-Marne  78 - Yvelines  91 - Essonne  \
0                 32.0                 5642           3146          2523   

   92 - Hauts-de-Seine  93001 - Aubervilliers  ...  73 - Savoie  \
0                167.0                    5.0  ...        13721   

   74 - Haute-Savoie  04 - Alpes-de-Haute-Provence  05 - Hautes-Alpes  \
0              12671                          5711               5625   

   06 - Alpes-Maritimes  13 - Bouches-du-Rhône  83 - Var  84 - Vaucluse  \
0                 10245                  17743     19069          15042   

   2A - Corse-du-Sud  2B - Haute-Corse  
0               3605              5279  

[1 rows x 179 columns]
   971 - Guadeloupe

In [44]:
import re

# Fonction pour supprimer les colonnes en fonction de la longueur des chiffres dans les en-têtes
def remove_columns_by_length(df):
    pattern = re.compile(r'^\d+')  # Recherche des en-têtes commençant par des chiffres
    filtered_columns = [col for col in df.columns if len(pattern.match(col.split(' - ')[0]).group()) > 3]
    return df.drop(columns=filtered_columns)

# Appliquer la fonction pour supprimer les colonnes indésirables des DataFrames
exploitations_data_cleaned = remove_columns_by_length(exploitations_data)
superficie_data_cleaned = remove_columns_by_length(superficie_data)

# Fonction pour séparer le tableau par année de référence et créer des variables dynamiques
def separate_by_year_and_create_vars(df, prefix):
    years = df.columns.str.split('/').str[0].unique()
    separated_dfs = {year: df.filter(like=year) for year in years}
    for year, df in separated_dfs.items():
        df.attrs['name'] = f"{prefix}_{year}"  # Utiliser un attribut personnalisé pour stocker le nom
        globals()[f"{prefix}_{year}"] = df
    return separated_dfs

# Séparer chaque tableau par année de référence et créer des variables dynamiques
exploitations_by_year_cleaned = separate_by_year_and_create_vars(exploitations_data_cleaned, "exploitations")
superficie_by_year_cleaned = separate_by_year_and_create_vars(superficie_data_cleaned, "superficie")

# Appliquer la fonction remove_indicator aux sous-tableaux par année pour enlever les dates
for year in exploitations_by_year_cleaned:
    exploitations_by_year_cleaned[year] = remove_indicator(exploitations_by_year_cleaned[year], year)

for year in superficie_by_year_cleaned:
    superficie_by_year_cleaned[year] = remove_indicator(superficie_by_year_cleaned[year], year)

# Fonction pour afficher le contenu des dictionnaires
def display_dict_content(dictionary, name):
    print(f"\nContenu du dictionnaire {name} :")
    for key, df in dictionary.items():
        print(f"\nClé : {key}")
        print(f"Nom du DataFrame : {df.attrs['name']}")  # Récupérer le nom depuis l'attribut personnalisé

# Afficher le contenu des dictionnaires
display_dict_content(exploitations_by_year_cleaned, "exploitations_by_year_cleaned")
display_dict_content(superficie_by_year_cleaned, "superficie_by_year_cleaned")

# Exemple d'accès à un DataFrame en dehors du dictionnaire
print("\nAccès direct à un DataFrame :")
print(exploitations_1970.head())
print(superficie_1970.head())



Contenu du dictionnaire exploitations_by_year_cleaned :

Contenu du dictionnaire superficie_by_year_cleaned :

Accès direct à un DataFrame :
   971 - Guadeloupe  972 - Martinique  973 - Guyane  974 - La Réunion  \
0               NaN               NaN           NaN               NaN   

   75 - Ville de Paris  77 - Seine-et-Marne  78 - Yvelines  91 - Essonne  \
0                 32.0                 5642           3146          2523   

   92 - Hauts-de-Seine  93001 - Aubervilliers  ...  73 - Savoie  \
0                167.0                    5.0  ...        13721   

   74 - Haute-Savoie  04 - Alpes-de-Haute-Provence  05 - Hautes-Alpes  \
0              12671                          5711               5625   

   06 - Alpes-Maritimes  13 - Bouches-du-Rhône  83 - Var  84 - Vaucluse  \
0                 10245                  17743     19069          15042   

   2A - Corse-du-Sud  2B - Haute-Corse  
0               3605              5279  

[1 rows x 179 columns]
   971 - Guadeloupe